In [5]:
import ray
ray.init(ignore_reinit_error = True)
import time

@ray.remote
class A:

    def test1(self):
        time.sleep(1)
        return "1"

    def test2(self):
        time.sleep(5)
        return "ok"

    def test3(self):
        time.sleep(20)
        return "test"

cl = A.remote()
processing_ids = [cl.test1.remote(), cl.test2.remote(), cl.test3.remote()]

while len(processing_ids):
    done_id, processing_ids = ray.wait(processing_ids)

    print(ray.get(done_id))
    print(f"In progress: {processing_ids}")


2022-03-08 15:35:32,015	INFO worker.py:862 -- Calling ray.init() again after it has already been called.


['1']
In progress: [ObjectRef(480a853c2c4c6f27891e2ac898ae26e07c69db8b0100000001000000), ObjectRef(623b26bdd75b28e9891e2ac898ae26e07c69db8b0100000001000000)]
['ok']
In progress: [ObjectRef(623b26bdd75b28e9891e2ac898ae26e07c69db8b0100000001000000)]
['test']
In progress: []


In [43]:
import time 
import random 
import ray 

ray.init(ignore_reinit_error = True) 

@ray.remote 
def do_some_work(x): 
    time.sleep(random.uniform(0, 4)) # Replace this with work you need to do. 
    return x 

def process_incremental(sum, result): 
    time.sleep(1) # Replace this with some processing code. 
    return sum + result 

start = time.time() 

result_ids = [do_some_work.remote(x) for x in range(4)] 
sum = 0 

while len(result_ids): 
    done_id, result_ids = ray.wait(result_ids)
    print(f"Dones: {done_id}")
    print(f"In progesss: {result_ids}")
    sum = process_incremental(sum, ray.get(done_id[0])) 

print("duration =", time.time() - start, "\nresult = ", sum) 

2022-03-08 14:46:31,370	INFO worker.py:862 -- Calling ray.init() again after it has already been called.


Dones: [ObjectRef(909a212b104ea2f1ffffffffffffffffffffffff0100000001000000)]
In progesss: [ObjectRef(8e088f779f48acd6ffffffffffffffffffffffff0100000001000000), ObjectRef(65a1a3aaa614cba3ffffffffffffffffffffffff0100000001000000), ObjectRef(5497aa04f981e4a1ffffffffffffffffffffffff0100000001000000)]
Dones: [ObjectRef(5497aa04f981e4a1ffffffffffffffffffffffff0100000001000000)]
In progesss: [ObjectRef(8e088f779f48acd6ffffffffffffffffffffffff0100000001000000), ObjectRef(65a1a3aaa614cba3ffffffffffffffffffffffff0100000001000000)]
Dones: [ObjectRef(65a1a3aaa614cba3ffffffffffffffffffffffff0100000001000000)]
In progesss: [ObjectRef(8e088f779f48acd6ffffffffffffffffffffffff0100000001000000)]
Dones: [ObjectRef(8e088f779f48acd6ffffffffffffffffffffffff0100000001000000)]
In progesss: []
duration = 5.368376970291138 
result =  6


In [ ]:
import ray
import time

# Start Ray.
# ray.init()

@ray.remote
def f(x):
    time.sleep(1)
    return x

# Start 4 tasks in parallel.
result_ids = []
for i in range(4):
    result_ids.append(f.remote(i))
    
# Wait for the tasks to complete and retrieve the results.
# With at least 4 cores, this will take 1 second.
results = ray.get(result_ids)  # [0, 1, 2, 3]

In [ ]:
import numpy as np

@ray.remote
def create_matrix(size):
    return np.random.normal(size=size)

@ray.remote
def multiply_matrices(x, y):
    return np.dot(x, y)

x_id = create_matrix.remote([1000, 1000])
y_id = create_matrix.remote([1000, 1000])
z_id = multiply_matrices.remote(x_id, y_id)

# Get the results.
z = ray.get(z_id)

In [ ]:
import numpy as np

def create_matrix(size):
    return np.random.normal(size=size)

def multiply_matrices(x, y):
    return np.dot(x, y)

x_id = create_matrix([1000, 1000])
y_id = create_matrix([1000, 1000])
z_id = multiply_matrices(x_id, y_id)

# Get the results.
z = (z_id)

In [ ]:

import time


@ray.remote
class MessageActor(object):
    def __init__(self):
        self.messages = []
    
    def add_message(self, message):
        self.messages.append(message)
    
    def get_and_clear_messages(self):
        messages = self.messages
        self.messages = []
        return messages


# Define a remote function which loops around and pushes
# messages to the actor.
@ray.remote
def worker(message_actor, j):
    for i in range(100):
        time.sleep(1)
        message_actor.add_message.remote(
            "Message {} from worker {}.".format(i, j))


# Create a message actor.
message_actor = MessageActor.remote()

# Start 3 tasks that push messages to the actor.
[worker.remote(message_actor, j) for j in range(3)]

# Periodically get the messages and print them.
for _ in range(20):
    new_messages = ray.get(message_actor.get_and_clear_messages.remote())
    print("New messages:", new_messages)
    time.sleep(1)

# This script prints something like the following:
# New messages: []
# New messages: ['Message 0 from worker 1.', 'Message 0 from worker 0.']
# New messages: ['Message 0 from worker 2.', 'Message 1 from worker 1.', 'Message 1 from worker 0.', 'Message 1 from worker 2.']
# New messages: ['Message 2 from worker 1.', 'Message 2 from worker 0.', 'Message 2 from worker 2.']
# New messages: ['Message 3 from worker 2.', 'Message 3 from worker 1.', 'Message 3 from worker 0.']
# New messages: ['Message 4 from worker 2.', 'Message 4 from worker 0.', 'Message 4 from worker 1.']
# New messages: ['Message 5 from worker 2.', 'Message 5 from worker 0.', 'Message 5 from worker 1.']

New messages: []
New messages: []
New messages: ['Message 0 from worker 0.', 'Message 0 from worker 2.', 'Message 0 from worker 1.']
New messages: ['Message 1 from worker 1.', 'Message 1 from worker 2.', 'Message 1 from worker 0.', 'Message 2 from worker 2.', 'Message 2 from worker 1.', 'Message 2 from worker 0.']
New messages: []
New messages: ['Message 3 from worker 0.', 'Message 3 from worker 1.', 'Message 3 from worker 2.', 'Message 4 from worker 2.', 'Message 4 from worker 1.']
New messages: ['Message 4 from worker 0.']
New messages: ['Message 5 from worker 2.', 'Message 5 from worker 1.', 'Message 5 from worker 0.']
New messages: ['Message 6 from worker 0.', 'Message 6 from worker 2.', 'Message 6 from worker 1.']
New messages: ['Message 7 from worker 0.', 'Message 7 from worker 2.', 'Message 7 from worker 1.']
New messages: ['Message 8 from worker 2.', 'Message 8 from worker 0.', 'Message 8 from worker 1.']
New messages: ['Message 9 from worker 1.', 'Message 9 from worker 2.', 'M

In [ ]:
from athena.src.services.download.data import Data

rules = {
    "domains": ["stocks", "variable etfs", "fixed etfs"]
}

backtesting_years = 3
download = Data(rules = rules, period = f"{backtesting_years + 2}y")
data = download.get_data(yahoo = True)

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from athena.src.services.backtest.backtest import Backtest
import ray

@ray.remote
def backtest(fix_percentage: float, data: pd.DataFrame, rebalance: str) -> pd.DataFrame:
    filters = {

        "max_price": {
            "cutoff": 70.0
        },

        "volume": {
            "num_tickers": 50,
            "months": 1
        },

        "momentum": {
            "num_tickers": 8,
            "months": 1
        }

    }
    
    if fix_percentage == 0:
        constraints = None

    else:
        constraints = {
            "invest_sections": {fix_percentage: ["FIXA11.SA", "IRFM11.SA"]}

        }


    optimizer = {
        "hrp": {
            "covariance_function": "exp_cov",
            "linkage_method": "average"
        }
    }

    backtest = Backtest(data_rules = rules, filters = filters, optimizer = optimizer, backtesting_years = backtesting_years, rebalance_interval = rebalance, data = data, constraints = constraints)
    backtest.backtest()
    
    return backtest.returns

In [ ]:
percentages = [0.25, 0.5, 0.75, 0, (0, 0.8)]
cons_future = backtest.remote(fix_percentage = 0.75, data = data)
mode_future = backtest.remote(fix_percentage = 0.5, data = data)
agre_future = backtest.remote(fix_percentage = 0.25, data = data)
vari_future = backtest.remote(fix_percentage = 0, data = data)
dyna_future = backtest.remote(fix_percentage = (0, 0.8), data = data)

cons = ray.get(cons_future)
mode = ray.get(mode_future)
agre = ray.get(agre_future)
vari = ray.get(vari_future)
dyna = ray.get(dyna_future)

In [4]:
import ray
ray.init(ignore_reinit_error = True)
import time
from datetime import datetime
import pandas as pd

2022-05-30 15:55:22,323	INFO worker.py:862 -- Calling ray.init() again after it has already been called.


In [24]:
def get_strategy() -> dict:
    strategy = datetime.now()
    return strategy


def run_strategy(strategy) -> pd.DataFrame:
    df_returns = pd.DataFrame(0, columns = [strategy], index = ["teste"])
    time.sleep(2)
    return df_returns

@ray.remote
def run_partial_universe(num) -> pd.DataFrame:
    main_df = pd.DataFrame()

    for i in range(num):
        random_strategy = get_strategy()

        df = run_strategy(strategy = random_strategy)
        main_df = pd.concat([main_df, df], axis = 1)

    return df

In [33]:
main_df = pd.DataFrame()

future_dfs = [run_partial_universe.remote(num = 4) for i in range(8)]

dfs = [df for df in ray.get(future_dfs)]

for df in dfs:
    main_df = pd.concat([main_df, df], axis = 1)

main_df

,2022-05-30 16:09:43.323403,2022-05-30 16:09:43.323403,2022-05-30 16:09:43.323403,2022-05-30 16:09:43.322395,2022-05-30 16:09:43.323403,2022-05-30 16:09:43.322395,2022-05-30 16:09:43.323403,2022-05-30 16:09:43.323403
teste,0,0,0,0,0,0,0,0


In [35]:
len(main_df.columns)

8